In [15]:
import sys, os, time

import open3d as o3d
import trimesh
import openmesh as om

import cv2

import scipy as sp
from scipy.sparse.linalg import eigs
from scipy.sparse import csr_matrix, coo_matrix, identity
from scipy.sparse.linalg import spsolve
from scipy.spatial import Delaunay, Voronoi

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

%matplotlib inline

In [16]:
# Tests of Openmesh
# https://gitlab.vci.rwth-aachen.de:9000/OpenMesh/openmesh-python/tree/master/tests

In [17]:
def read_mesh_om(path_in_models):
    return om.read_trimesh(os.path.join('..', 'models', path_in_models))
def write_mesh_om(mesh, path_in_models):
    om.write_mesh(os.path.join('..', 'models', path_in_models), mesh, vertex_color=True)
def show_mesh_o3d(plys):
    o3d.visualization.draw_geometries(plys)
def write_mesh_o3d(path, mesh):
    o3d.io.write_triangle_mesh(path, mesh)
def read_mesh_o3d(mesh_fp):
    return o3d.io.read_triangle_mesh(mesh_fp)
def read_mesh_trimesh(path_in_models):
    return trimesh.load(os.path.join('..', 'models', path_in_models))

### Calculate Voronoi Poles

In [18]:
INFINITY = 1e9
EPS = 1e-9
def get_vertices_from_trimesh(mesh):
    return np.asarray(mesh.vertices)

def get_vertice_normals_from_trimesh(mesh):
    return np.asarray(mesh.vertex_normals)

def calculate_voronoi_poles(mesh):
    
    vertices = get_vertices_from_trimesh(mesh)
    vertice_normals = get_vertice_normals_from_trimesh(mesh)

    print("Calculating Voronoi Diagram.")
    vor = Voronoi(vertices)
    vor_centers = vor.vertices
    cells = vor.regions
    cell_indices = vor.point_region

    vertices_total = vertices.shape[0]
    voronoi_poles = np.zeros(vertices.shape)
    print("Calculating the Voronoi Pole for each vertex.")
    for vi in tqdm(range(vertices_total)):
        vor_cell = cells[cell_indices[vi]]
        vertice = vertices[vi]
        vertice_normal = vertice_normals[vi]
        max_neg_proj = INFINITY
        voronoi_pole = None
        for vci in vor_cell:
            vor_center = vor_centers[vci]
            if vci == -1:
                continue
            proj = np.dot(vor_center - vertice, vertice_normal)
            if proj < max_neg_proj:
                max_neg_proj = proj
                voronoi_pole = vor_center
        voronoi_poles[vi] = voronoi_pole

    return voronoi_poles

### Calculate Cotangent Laplace Operator

In [28]:
def trimesh_Generate_Area_List(mesh):

    area_list = np.zeros((len(mesh.vertices)))
    face_count = mesh.faces.shape[0]
    face_area = np.asarray(mesh.area_faces)
    faces = np.asarray(mesh.faces)
    for i in range(face_count):
        area_list[faces[i]] += face_area[i] / 3

    return area_list


def trimesh_Generate_Laplace_matrix(mesh):
    
    print('Constructing Laplace Matrix.')
    
    vertices_face_indexs = [[0,1,2],[1,0,2],[2,0,1]] 
    laplace_dict = {}
    area_list = trimesh_Generate_Area_List(mesh)
    face_angles = np.asarray(mesh.face_angles)
    faces = np.asarray(mesh.faces)
    face_count = faces.shape[0]
    
    with tqdm(total=face_count) as tbar:
        for face, angles in zip(faces, face_angles):
            for i in range(3):
                current_angle = angles[i]
                v0_index, v1_index, v2_index = face[vertices_face_indexs[i]]
                delta = 0.5 / area_list[v0_index] / np.tan(current_angle)
                if delta > EPS:
                    laplace_dict[(v1_index,v1_index)] = laplace_dict.get((v1_index,v1_index),0) - delta
                    laplace_dict[(v2_index,v2_index)] = laplace_dict.get((v2_index,v2_index),0) - delta
                    laplace_dict[(v1_index,v2_index)] = laplace_dict.get((v1_index,v2_index),0) + delta
                    laplace_dict[(v2_index,v1_index)] = laplace_dict.get((v2_index,v1_index),0) + delta
            tbar.update(1)

    # Construct CSR Matrix
    rows, cols = zip(*laplace_dict.keys())
    values = list(laplace_dict.values())
    coo = coo_matrix((values, (rows, cols)), shape=(len(mesh.vertices), len(mesh.vertices)))
    csr = coo.tocsr()

    return csr

# Testing Blocks

In [29]:
mesh = read_mesh_trimesh('armadillo.obj')
voronoi_poles = calculate_voronoi_poles(mesh)
laplace_csr = trimesh_Generate_Laplace_matrix(mesh)

Calculating Voronoi Diagram.
Calculating the Voronoi Pole for each vertex.


100%|██████████| 25193/25193 [00:00<00:00, 28284.34it/s]


Constructing Laplace Matrix.


100%|██████████| 50382/50382 [00:00<00:00, 80276.49it/s]
